In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from bayes_opt import BayesianOptimization, UtilityFunction
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Prepare the data.
cancer = load_breast_cancer()
X = cancer["data"]
y = cancer["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                            stratify = y,
                                        random_state = 42)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Define the black box function to optimize.
def black_box_function(C):
    # C: SVC hyper parameter to optimize for.
    model = SVC(C = C)
    model.fit(X_train_scaled, y_train)
    y_score = model.decision_function(X_test_scaled)
    f = roc_auc_score(y_test, y_score)
    return f
# Set range of C to optimize for.
# bayes_opt requires this to be a dictionary.
pbounds = {"C": [0.1, 10]}
# Create a BayesianOptimization optimizer,
# and optimize the given black_box_function.
optimizer = BayesianOptimization(f = black_box_function,
                                 pbounds = pbounds, verbose = 2,
                                 random_state = 4)
optimizer.maximize(init_points = 5, n_iter = 10)
print("Best result: {}; f(x) = {}.".format(optimizer.max["params"], optimizer.max["target"]))

|   iter    |  target   |     C     |
-------------------------------------
| 1         | 0.9979    | 9.674     |
| 2         | 0.9975    | 5.518     |
| 3         | 0.9979    | 9.73      |
| 4         | 0.9979    | 7.177     |
| 5         | 0.9979    | 7.008     |
| 6         | 0.9914    | 0.1023    |
| 7         | 0.9981    | 8.506     |
| 8         | 0.9981    | 8.15      |
| 9         | 0.9981    | 8.327     |
| 10        | 0.9981    | 8.8       |
| 11        | 0.9981    | 8.671     |
| 12        | 0.9981    | 7.974     |
| 13        | 0.9979    | 6.273     |
| 14        | 0.9981    | 8.064     |
| 15        | 0.9981    | 8.911     |
Best result: {'C': 8.50571739015795}; f(x) = 0.9981132075471698.


1D Linear System

In [4]:
N = 30
x = np.zeros((N+1,1))
u = np.zeros((N,1))
R = 1
Q = 1
Qf = 1
D = np.zeros((2,))
theta = np.ones((2,))

search_space = {}
search_space["A"]=(-5,5)
search_space["B"]=(-5,5)

# define transition function (parametrized)
def fx(zk, uk, A, B):
    return A*zk + B*uk

# define cost functional (reversed)
def J(z,u, N=N):
    sum = 0
    for k in range(N) :
        sum += z[k]*Q*z[k] + u[k]*Q*u[k]
    return -1*(sum + z[N]*Qf*z[N])

# set initial condition
t=0
x[0,:] = 0
u[0,:] = 0
x_list = []
u_list = []

# compute time evolution with current control sequence and current parametrization
for k in range(N):
    x[k+1,:] = fx(x[k,:], u[k,:], theta[0], theta[1])

def objective(A, B):
    for k in range(N):
        x[k+1,:] = fx(x[k,:], u[k,:], A, B)
    return J(x, u)

optimizer = BayesianOptimization(
                                f=objective,
                                pbounds=search_space,
                                random_state=1
                              )
utility = UtilityFunction(kind="ucb", kappa=2.5, xi=0.0)

def find_control_sequence(i, N, theta):
    u_hat = np.zeros((N-i),1)
    
    return u_hat

optimize=True
iter = 0
nit_max = 100
while optimize:
    iter += 1
    theta[0] = optimizer.suggest(utility)["A"]
    theta[1] = optimizer.suggest(utility)["B"]
    x_list = [x[0,:]]
    u_list = []
    for i in range(N-1):
        pi = find_control_sequence(i, N, theta)
        x_new = fx(x_list[-1], pi[0])
        x_list.append(x_new)
        u_list.append(pi[0])

    if iter>nit_max:
        optimize = False





optimizer.maximize()
